1. 이미지분석
2. custom vision- clssification
3. face 
4. OCR
5. Video Indexer

# Azure AI Vision 솔루션

* .NET 패키지 설치

In [20]:
//#r "nuget: Azure.AI.Vision.ImageAnalysis, 1.0.0-beta.1"
#r "nuget: Microsoft.Extensions.Configuration.Json, 3.1.3"
#r "nuget: System.Drawing.Common, 4.7.0"
#r "nuget: Microsoft.Azure.CognitiveServices.Vision.ComputerVision"
#r "nuget: Azure.AI.Vision.Common"
#r "nuget: Microsoft.Azure.CognitiveServices.Vision.Face, 2.8.0-preview.3"
#r "nuget: Azure.AI.Vision.Core, 0.10.0-beta.1"
#r "nuget: Azure.AI.Vision.ImageAnalysis, 1.0.0-beta.2"
#r "nuget: Microsoft.Azure.CognitiveServices.Language.LUIS.Runtime, 3.0.0"
#r "nuget: Microsoft.Azure.CognitiveServices.Language.TextAnalytics, 3.0.0"

Installed Packages Azure.AI.Vision.Common, 0.11.0 Azure.AI.Vision.Core, 0.10.0-beta.1 Azure.AI.Vision.ImageAnalysis, 1.0.0-beta.2 Microsoft.Azure.CognitiveServices.Language.LUIS.Runtime, 3.0.0 Microsoft.Azure.CognitiveServices.Language.TextAnalytics, 3.0.0 Microsoft.Azure.CognitiveServices.Vision.ComputerVision, 7.0.1 Microsoft.Azure.CognitiveServices.Vision.Face, 2.8.0-preview.3 Microsoft.Extensions.Configuration.Json, 3.1.3 System.Drawing.Common, 4.7.0

Error: C:\Users\parkm\.packagemanagement\nuget\Projects\39632--5860451e-8234-4506-a31c-42b270b4284d\Project.fsproj : error NU1101: Microsoft.Azure.CognitiveServices.Authentications 패키지를 찾을 수 없습니다. 원본에 ID가 C:\Program Files\dotnet\library-packs, C:\Program Files\dotnet\sdk\8.0.205\FSharp\library-packs, nuget.org인 패키지가 없습니다.
C:\Users\parkm\.packagemanagement\nuget\Projects\39632--5860451e-8234-4506-a31c-42b270b4284d\Project.fsproj : warning NU1902: 'Microsoft.Rest.ClientRuntime' 2.3.20 패키지에 알려진 보통 심각도 취약성인 https://github.com/advisories/GHSA-whph-446h-6m9v이(가) 있습니다.
C:\Users\parkm\.packagemanagement\nuget\Projects\39632--5860451e-8234-4506-a31c-42b270b4284d\Project.fsproj : warning NU1903: 'Newtonsoft.Json' 10.0.3 패키지에 알려진 높은 심각도 취약성인 https://github.com/advisories/GHSA-5crp-9r3c-p9vr이(가) 있습니다.
C:\Users\parkm\.packagemanagement\nuget\Projects\39632--5860451e-8234-4506-a31c-42b270b4284d\Project.fsproj : warning NU1904: 'System.Drawing.Common' 4.7.0 패키지에 알려진 위험 심각도 취약성인 https://github.com/

* namespace 가져오기

In [21]:
using System;
using System.IO;
using System.Net.Http;
using System.Net.Http.Headers;
using System.Text;
using System.Text.Json;
using System.Threading.Tasks;
using System.Drawing;
using Microsoft.Extensions.Configuration;
using Azure;
using Azure.AI.Vision.ImageAnalysis;
using Azure.AI.Vision.Common;
using Azure.AI.Vision;
using Microsoft.Azure.CognitiveServices.Vision.Face;
using Microsoft.Azure.CognitiveServices.Vision.Face.Models;
using Microsoft.Azure.CognitiveServices.Vision.ComputerVision;
using Microsoft.Azure.CognitiveServices.Vision.ComputerVision.Models;
using Azure.AI.Vision.Core.Options;
using Azure.AI.Vision.Core.Input;
using System.Linq;
using System.Collections.Generic;
using System.Threading;
using Microsoft.Azure.CognitiveServices.Language.LUIS.Runtime;


# 1. 이미지 분석

In [3]:
public class image_analysis
    {

        public static async Task Main(string[] args)
        {
            try
            {
                // Get config settings from AppSettings
                String address_1="C:\\Users\\parkm\\Desktop\\Vision\\mslearn-ai-vision\\Labfiles\\01-analyze-images\\C-Sharp\\image-analysis\\";
                
                IConfigurationBuilder builder = new ConfigurationBuilder().AddJsonFile(address_1+"appsettings.json");
                IConfigurationRoot configuration = builder.Build();
                string aiSvcEndpoint = configuration["AIServicesEndpoint"];
                string aiSvcKey = configuration["AIServicesKey"];

                // Get image
                string imageFile = address_1+"images/street.jpg";
                if (args.Length > 0)
                {
                    imageFile = args[0];
                }

                // Authenticate Azure AI Vision client
                var client = new ImageAnalysisClient(new Uri(aiSvcEndpoint), new AzureKeyCredential(aiSvcKey));
                
                // Analyze image
                AnalyzeImage(imageFile, client);

                // Remove the background or generate a foreground matte from the image
                await BackgroundForeground(imageFile, aiSvcEndpoint, aiSvcKey);

            }
            catch (Exception ex)
            {
                Console.WriteLine(ex.Message);
            }
        }

        static void AnalyzeImage(string imageFile, ImageAnalysisClient client)
        {
            Console.WriteLine($"\nAnalyzing {imageFile} \n");

            // Use a file stream to pass the image data to the analyze call
            using FileStream stream = new FileStream(imageFile,
                                                     FileMode.Open);

            // Get result with specified features to be retrieved
            ImageAnalysisResult result = client.Analyze(
                BinaryData.FromStream(stream),
                VisualFeatures.Caption | 
                VisualFeatures.DenseCaptions |
                VisualFeatures.Objects |
                VisualFeatures.Tags |
                VisualFeatures.People);
                        
            // Display analysis results
            // Get image captions
            if (result.Caption.Text != null)
            {
                Console.WriteLine(" Caption:");
                Console.WriteLine($"   \"{result.Caption.Text}\", Confidence {result.Caption.Confidence:0.00}\n");
            }

            // Get image dense captions
            Console.WriteLine(" Dense Captions:");
            foreach (DenseCaption denseCaption in result.DenseCaptions.Values)
            {
                Console.WriteLine($"   Caption: '{denseCaption.Text}', Confidence: {denseCaption.Confidence:0.00}");
            }

        }
        static async Task BackgroundForeground(string imageFile, string endpoint, string key)
        {
            // Remove the background from the image or generate a foreground matte
            Console.WriteLine($" Background removal:");
            // Define the API version and mode
            string apiVersion = "2023-02-01-preview";
            string mode = "backgroundRemoval"; // Can be "foregroundMatting" or "backgroundRemoval"

            string url = $"computervision/imageanalysis:segment?api-version={apiVersion}&mode={mode}";

            // Make the REST call
            using (var client = new HttpClient())
            {
                var contentType = new MediaTypeWithQualityHeaderValue("application/json");
                client.BaseAddress = new Uri(endpoint);
                client.DefaultRequestHeaders.Accept.Add(contentType);
                client.DefaultRequestHeaders.Add("Ocp-Apim-Subscription-Key", key);

                var data = new
                {
                    url = $"https://github.com/MicrosoftLearning/mslearn-ai-vision/blob/main/Labfiles/01-analyze-images/Python/image-analysis/images/street.jpg?raw=true"
                };

                var jsonData = JsonSerializer.Serialize(data);
                var contentData = new StringContent(jsonData, Encoding.UTF8, contentType);
                var response = await client.PostAsync(url, contentData);

                if (response.IsSuccessStatusCode) {
                    File.WriteAllBytes("background.png", response.Content.ReadAsByteArrayAsync().Result);
                    Console.WriteLine("  Results saved in background.png\n");
                }
                else
                {
                    Console.WriteLine($"API error: {response.ReasonPhrase} - Check your body url, key, and endpoint.");
                }
            }
            
        }
    }



In [4]:
await image_analysis.Main(new string[0]);


Analyzing C:\Users\parkm\Desktop\Vision\mslearn-ai-vision\Labfiles\01-analyze-images\C-Sharp\image-analysis\images/street.jpg 

 Caption:
   "a man walking a dog on a leash on a street", Confidence 0.82

 Dense Captions:
   Caption: 'a man walking a dog on a leash on a street', Confidence: 0.82
   Caption: 'a man walking on a street', Confidence: 0.69
   Caption: 'a yellow car on the street', Confidence: 0.78
   Caption: 'a black dog walking on the street', Confidence: 0.75
   Caption: 'a blurry image of a blue car', Confidence: 0.82
   Caption: 'a yellow taxi cab on the street', Confidence: 0.72
 Background removal:
  Results saved in background.png



# 3.얼굴 감지

* Computer Vision

In [5]:
public class detect_people
{
        public static void Main(string[] args)
        {
            try
            {
                // Get config settings from AppSettings
                String address_2="C:/Users/parkm/Desktop/Vision/mslearn-ai-vision/Labfiles/04-face/C-Sharp/computer-vision/";
                string aiSvcEndpoint = "https://fruit-train.cognitiveservices.azure.com/";
                string aiSvcKey = "68309624561e4968~~";

                // Get image
                string imageFile = address_2+"images/people.jpg";
                if (args.Length > 0)
                {
                    imageFile = args[0];
                }

                // Authenticate Azure AI Vision client
                var cvClient = new VisionServiceOptions(
                aiSvcEndpoint,
                new AzureKeyCredential(aiSvcKey));

                
                // Analyze image
                AnalyzeImage(imageFile, cvClient);

            }
            catch (Exception ex)
            {
                Console.WriteLine(ex.Message);
            }
        }

        static void AnalyzeImage(string imageFile, VisionServiceOptions serviceOptions)
        {
            Console.WriteLine($"\nAnalyzing {imageFile} \n");

            var analysisOptions = new ImageAnalysisOptions()
            {
                 // Specify features to be retrieved (PEOPLE)
                Features =
                     ImageAnalysisFeature.People
            };

            // Get image analysis
            using var imageSource = VisionSource.FromFile(imageFile);
                
            using var analyzer = new ImageAnalyzer(serviceOptions, imageSource, analysisOptions);
                
            var result = analyzer.Analyze();
                
            if (result.Reason == ImageAnalysisResultReason.Analyzed)
            {
                // Get people in the image
                if (result.People != null)
                {
                    Console.WriteLine($" People:");
                    
                    // Prepare image for drawing
                    System.Drawing.Image image = System.Drawing.Image.FromFile(imageFile);
                    Graphics graphics = Graphics.FromImage(image);
                    Pen pen = new Pen(Color.Cyan, 3);
                    Font font = new Font("Arial", 16);
                    SolidBrush brush = new SolidBrush(Color.WhiteSmoke);
                    
                    foreach (var person in result.People)
                    {
                        // Draw object bounding box if confidence > 50%
                        if (person.Confidence > 0.5)
                        {
                            // Draw object bounding box
                            var r = person.BoundingBox;
                            Rectangle rect = new Rectangle(r.X, r.Y, r.Width, r.Height);
                            graphics.DrawRectangle(pen, rect);
                    
                            // Return the confidence of the person detected
                            Console.WriteLine($"   Bounding box {person.BoundingBox}, Confidence {person.Confidence:0.0000}");
                        }
                    }
                    
                    // Save annotated image
                    String output_file = "detected_people.jpg";
                    image.Save(output_file);
                    Console.WriteLine("  Results saved in " + output_file + "\n");
                }
            }
            else
            {
                var errorDetails = ImageAnalysisErrorDetails.FromResult(result);
                Console.WriteLine(" Analysis failed.");
                Console.WriteLine($"   Error reason : {errorDetails.Reason}");
                Console.WriteLine($"   Error code : {errorDetails.ErrorCode}");
                Console.WriteLine($"   Error message: {errorDetails.Message}\n");
            }
 
        }
}


Error: (42,17): error CS0117: 'ImageAnalysisOptions'에는 'Features'에 대한 정의가 포함되어 있지 않습니다.
(43,22): error CS0103: 'ImageAnalysisFeature' 이름이 현재 컨텍스트에 없습니다.
(49,38): error CS0246: 'ImageAnalyzer' 형식 또는 네임스페이스 이름을 찾을 수 없습니다. using 지시문 또는 어셈블리 참조가 있는지 확인하세요.
(53,34): error CS0103: 'ImageAnalysisResultReason' 이름이 현재 컨텍스트에 없습니다.
(90,36): error CS0103: 'ImageAnalysisErrorDetails' 이름이 현재 컨텍스트에 없습니다.

In [ ]:
DetectPeople.Main(new string[] {});

* Face API

In [16]:
public class analyze_faces
{
        private static FaceClient faceClient;
        static async Task Main(string[] args)
        {
            try
            {
                // Get config settings from AppSettings

                string cogSvcEndpoint = "ttps://fruit-train.cognitiveservices.azure.com/";
                string cogSvcKey = "68309624561e4968~~";

                // Authenticate Face client
                var credentials = new ApiKeyServiceClientCredentials(cogSvcKey);
                faceClient = new FaceClient(credentials)
                {
                    Endpoint = cogSvcEndpoint
                };

                // Menu for face functions
                Console.WriteLine("1: Detect faces\nAny other key to quit");
                Console.WriteLine("Enter a number:");
                string command = Console.ReadLine();
                switch (command)
                {
                    case "1":
                        await DetectFaces("images/people.jpg");
                        break;
                    default:
                        break;
                }
            }
            catch (Exception ex)
            {
                Console.WriteLine(ex.Message);
            }
        }

        static async Task DetectFaces(string imageFile)
        {
            Console.WriteLine($"Detecting faces in {imageFile}");

            // Specify facial features to be retrieved
             List<VisualFeatureTypes?> features = new List<VisualFeatureTypes?>()
            {
                VisualFeatureTypes.Faces
            };

             // Get faces
            using (var imageData = File.OpenRead(imageFile))
            {    
                var detected_faces = await faceClient.Face.DetectWithStreamAsync(imageData, returnFaceAttributes: features, returnFaceId: false);

                if (detected_faces.Count > 0)
                {
                    Console.WriteLine($"{detected_faces.Count} faces detected.");

                    // Prepare image for drawing
                    Image image = Image.FromFile(imageFile);
                    Graphics graphics = Graphics.FromImage(image);
                    Pen pen = new Pen(Color.LightGreen, 3);
                    Font font = new Font("Arial", 4);
                    SolidBrush brush = new SolidBrush(Color.Black);
                    int faceCount=0;

                    // Draw and annotate each face
                    foreach (var face in detected_faces)
                    {
                        faceCount++;
                        Console.WriteLine($"\nFace number {faceCount}");
                        
                        // Get face properties
                        Console.WriteLine($" - Mouth Occluded: {face.FaceAttributes.Occlusion.MouthOccluded}");
                        Console.WriteLine($" - Eye Occluded: {face.FaceAttributes.Occlusion.EyeOccluded}");
                        Console.WriteLine($" - Blur: {face.FaceAttributes.Blur.BlurLevel}");
                        Console.WriteLine($" - Glasses: {face.FaceAttributes.Glasses}");

                        // Draw and annotate face
                        var r = face.FaceRectangle;
                        Rectangle rect = new Rectangle(r.Left, r.Top, r.Width, r.Height);
                        graphics.DrawRectangle(pen, rect);
                        string annotation = $"Face ID: {face.FaceId}";
                        graphics.DrawString(annotation,font,brush,r.Left, r.Top);
                    }

                    // Save annotated image
                    String output_file = "detected_faces.jpg";
                    image.Save(output_file);
                    Console.WriteLine(" Results saved in " + output_file);   
                }
            }
 
 
        }
}



Error: (14,39): error CS0104: 'ApiKeyServiceClientCredentials'은(는) 'Microsoft.Azure.CognitiveServices.Vision.ComputerVision.ApiKeyServiceClientCredentials' 및 'Microsoft.Azure.CognitiveServices.Vision.Face.ApiKeyServiceClientCredentials' 사이에 모호한 참조입니다.
(52,115): error CS1503: 3 인수: 'System.Collections.Generic.List<Microsoft.Azure.CognitiveServices.Vision.ComputerVision.Models.VisualFeatureTypes?>'에서 'System.Collections.Generic.IList<Microsoft.Azure.CognitiveServices.Vision.Face.Models.FaceAttributeType>'(으)로 변환할 수 없습니다.
(54,21): error CS0019: '>' 연산자는 '메서드 그룹' 및 'int' 형식의 피연산자에 적용할 수 없습니다.
(56,42): error CS1503: 1 인수: '메서드 그룹'에서 'scoped System.ReadOnlySpan<char>'(으)로 변환할 수 없습니다.

# 4. OCR

In [6]:
 public static String address_3="C:\\Users\\parkm\\Desktop\\Vision\\mslearn-ai-vision\\Labfiles\\05-ocr\\C-Sharp\\read-text\\images\\";

In [7]:
public class read_text
    {

        public static void Main(string[] args)
        {
            try
            {
                // Get config settings from AppSettings
                string aiSvcEndpoint = "https://fruit-train.cognitiveservices.azure.com/";
                string aiSvcKey = "68309624561e4~~";

                // Authenticate Azure AI Vision client
                ImageAnalysisClient client = new ImageAnalysisClient(
                new Uri(aiSvcEndpoint),
                new AzureKeyCredential(aiSvcKey));

                // Menu for text reading functions
                Console.WriteLine("\n1: Use Read API for image (Lincoln.jpg)\n2: Read handwriting (Note.jpg)\nAny other key to quit\n");
                string imageFile;

                switch (args[0])
                {
                    case "1":
                        imageFile = address_3+"Lincoln.jpg";
                        GetTextRead(imageFile, client);
                        break;
                    case "2":
                        imageFile = address_3+"Note.jpg";
                        GetTextRead(imageFile, client);
                        break;
                    default:
                        break;
                }

            }
            catch (Exception ex)
            {
                Console.WriteLine(ex.Message);
            }
        }

        static void GetTextRead(string imageFile, ImageAnalysisClient client)
        {
            Console.WriteLine($"\nReading text from {imageFile} \n");

            // Use a file stream to pass the image data to the analyze call
            using FileStream stream = new FileStream(imageFile,
                                                     FileMode.Open);

            // Use Analyze image function to read text in image
            ImageAnalysisResult result = client.Analyze(
                BinaryData.FromStream(stream),
                // Specify the features to be retrieved
                VisualFeatures.Read);
                
            stream.Close();
                
            // Display analysis results
            if (result.Read != null)
            {
                Console.WriteLine($"Text:");
                
                // Prepare image for drawing
                System.Drawing.Image image = System.Drawing.Image.FromFile(imageFile);
                Graphics graphics = Graphics.FromImage(image);
                Pen pen = new Pen(Color.Cyan, 3);
                    
                foreach (var line in result.Read.Blocks.SelectMany(block => block.Lines))
                {
                     // Return the text detected in the image
                    Console.WriteLine($"   '{line.Text}'");
                        
                    // Draw bounding box around line
                    var drawLinePolygon = true;
                        
                    // Return the position bounding box around each line
                    Console.WriteLine($"   Bounding Polygon: [{string.Join(" ", line.BoundingPolygon)}]");   
                        
                   // Return each word detected in the image and the position bounding box around each word with the confidence level of each word
                    foreach (DetectedTextWord word in line.Words)
                    {
                        Console.WriteLine($"     Word: '{word.Text}', Confidence {word.Confidence:F4}, Bounding Polygon: [{string.Join(" ", word.BoundingPolygon)}]");
                            
                        // Draw word bounding polygon
                        drawLinePolygon = false;
                        var r = word.BoundingPolygon;
                        
                        Point[] polygonPoints = {
                            new Point(r[0].X, r[0].Y),
                            new Point(r[1].X, r[1].Y),
                            new Point(r[2].X, r[2].Y),
                            new Point(r[3].X, r[3].Y)
                        };
                        
                        graphics.DrawPolygon(pen, polygonPoints);
                    }
                        
                        
                        
                    // Draw line bounding polygon
                    if (drawLinePolygon)
                    {
                        var r = line.BoundingPolygon;
                        
                        Point[] polygonPoints = {
                            new Point(r[0].X, r[0].Y),
                            new Point(r[1].X, r[1].Y),
                            new Point(r[2].X, r[2].Y),
                            new Point(r[3].X, r[3].Y)
                        };
                        
                        graphics.DrawPolygon(pen, polygonPoints);
                    }
                                    
                
                }
                        
                // Save image
                String output_file = "text.jpg";
                image.Save(output_file);
                Console.WriteLine("\nResults saved in " + output_file + "\n");   
            }
            
    
        }
}


In [8]:
read_text.Main(new string[] { "2" });


1: Use Read API for image (Lincoln.jpg)
2: Read handwriting (Note.jpg)
Any other key to quit


Reading text from C:\Users\parkm\Desktop\Vision\mslearn-ai-vision\Labfiles\05-ocr\C-Sharp\read-text\images\Note.jpg 

Text:
   'Shopping List'
   Bounding Polygon: [(231, 141) (693, 147) (691, 245) (230, 240)]
     Word: 'Shopping', Confidence 0.9630, Bounding Polygon: [(240, 141) (535, 149) (531, 245) (234, 234)]
     Word: 'List', Confidence 0.8300, Bounding Polygon: [(554, 149) (689, 147) (686, 244) (550, 245)]
   'Non- Fat milk'
   Bounding Polygon: [(149, 302) (633, 297) (633, 374) (150, 378)]
     Word: 'Non-', Confidence 0.5770, Bounding Polygon: [(150, 303) (309, 301) (310, 378) (153, 378)]
     Word: 'Fat', Confidence 0.8420, Bounding Polygon: [(324, 301) (438, 300) (437, 378) (325, 378)]
     Word: 'milk', Confidence 0.9940, Bounding Polygon: [(476, 299) (620, 298) (617, 374) (475, 377)]
   'Bread'
   Bounding Polygon: [(138, 400) (382, 399) (383, 472) (138, 474)]
     Word: 'Bread